In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [2]:
import tensorflow as tf

In [3]:
# define the raw dataset
alphabet = "1234567890abcdefghijklmnopqrstuvwxyz"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	print(seq_in, '->', seq_out)

1 -> 2
2 -> 3
3 -> 4
4 -> 5
5 -> 6
6 -> 7
7 -> 8
8 -> 9
9 -> 0
0 -> a
a -> b
b -> c
c -> d
d -> e
e -> f
f -> g
g -> h
h -> i
i -> j
j -> k
k -> l
l -> m
m -> n
n -> o
o -> p
p -> q
q -> r
r -> s
s -> t
t -> u
u -> v
v -> w
w -> x
x -> y
y -> z


In [5]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

In [6]:
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
%%time
# create and fit the model
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, batch_size=1, verbose=0)

# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Model Accuracy: 62.86%
Wall time: 15.3 s


In [8]:
# demonstrate some model predictions
for pattern in dataX:
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(seq_in, "->", result)

['1'] -> 2
['2'] -> 2
['3'] -> 3
['4'] -> 4
['5'] -> 6
['6'] -> 7
['7'] -> 8
['8'] -> 8
['9'] -> 0
['0'] -> 0
['a'] -> b
['b'] -> c
['c'] -> d
['d'] -> e
['e'] -> f
['f'] -> g
['g'] -> h
['h'] -> i
['i'] -> j
['j'] -> k
['k'] -> l
['l'] -> m
['m'] -> n
['n'] -> o
['o'] -> p
['p'] -> q
['q'] -> s
['r'] -> t
['s'] -> u
['t'] -> v
['u'] -> x
['v'] -> z
['w'] -> z
['x'] -> z
['y'] -> z


# 3 chars window

In [9]:
seq_length = 3
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	print(seq_in, '->', seq_out)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), 1, seq_length))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

123 -> 4
234 -> 5
345 -> 6
456 -> 7
567 -> 8
678 -> 9
789 -> 0
890 -> a
90a -> b
0ab -> c
abc -> d
bcd -> e
cde -> f
def -> g
efg -> h
fgh -> i
ghi -> j
hij -> k
ijk -> l
jkl -> m
klm -> n
lmn -> o
mno -> p
nop -> q
opq -> r
pqr -> s
qrs -> t
rst -> u
stu -> v
tuv -> w
uvw -> x
vwx -> y
wxy -> z


In [12]:
%%time
# create and fit the model
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, batch_size=1, verbose=0)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 66.67%
Wall time: 15.7 s


In [13]:
# demonstrate some model predictions
for pattern in dataX:
	x = numpy.reshape(pattern, (1, 1, len(pattern)))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(seq_in, "->", result)

['1', '2', '3'] -> 4
['2', '3', '4'] -> 4
['3', '4', '5'] -> 6
['4', '5', '6'] -> 6
['5', '6', '7'] -> 8
['6', '7', '8'] -> 9
['7', '8', '9'] -> 0
['8', '9', '0'] -> a
['9', '0', 'a'] -> b
['0', 'a', 'b'] -> c
['a', 'b', 'c'] -> d
['b', 'c', 'd'] -> e
['c', 'd', 'e'] -> f
['d', 'e', 'f'] -> g
['e', 'f', 'g'] -> h
['f', 'g', 'h'] -> i
['g', 'h', 'i'] -> j
['h', 'i', 'j'] -> k
['i', 'j', 'k'] -> l
['j', 'k', 'l'] -> m
['k', 'l', 'm'] -> m
['l', 'm', 'n'] -> p
['m', 'n', 'o'] -> p
['n', 'o', 'p'] -> q
['o', 'p', 'q'] -> r
['p', 'q', 'r'] -> t
['q', 'r', 's'] -> u
['r', 's', 't'] -> x
['s', 't', 'u'] -> x
['t', 'u', 'v'] -> z
['u', 'v', 'w'] -> z
['v', 'w', 'x'] -> z
['w', 'x', 'y'] -> z


# Timesteps

In [14]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 3
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	print(seq_in, '->', seq_out)

123 -> 4
234 -> 5
345 -> 6
456 -> 7
567 -> 8
678 -> 9
789 -> 0
890 -> a
90a -> b
0ab -> c
abc -> d
bcd -> e
cde -> f
def -> g
efg -> h
fgh -> i
ghi -> j
hij -> k
ijk -> l
jkl -> m
klm -> n
lmn -> o
mno -> p
nop -> q
opq -> r
pqr -> s
qrs -> t
rst -> u
stu -> v
tuv -> w
uvw -> x
vwx -> y
wxy -> z


In [15]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [16]:
%%time
# create and fit the model
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, batch_size=1, verbose=0)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 96.97%
Wall time: 25 s


In [17]:
# demonstrate some model predictions
for pattern in dataX:
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(seq_in, "->", result)

['1', '2', '3'] -> 4
['2', '3', '4'] -> 5
['3', '4', '5'] -> 6
['4', '5', '6'] -> 7
['5', '6', '7'] -> 8
['6', '7', '8'] -> 9
['7', '8', '9'] -> 0
['8', '9', '0'] -> a
['9', '0', 'a'] -> b
['0', 'a', 'b'] -> c
['a', 'b', 'c'] -> d
['b', 'c', 'd'] -> e
['c', 'd', 'e'] -> f
['d', 'e', 'f'] -> g
['e', 'f', 'g'] -> h
['f', 'g', 'h'] -> i
['g', 'h', 'i'] -> j
['h', 'i', 'j'] -> k
['i', 'j', 'k'] -> l
['j', 'k', 'l'] -> m
['k', 'l', 'm'] -> n
['l', 'm', 'n'] -> o
['m', 'n', 'o'] -> p
['n', 'o', 'p'] -> q
['o', 'p', 'q'] -> r
['p', 'q', 'r'] -> s
['q', 'r', 's'] -> t
['r', 's', 't'] -> u
['s', 't', 'u'] -> v
['t', 'u', 'v'] -> w
['u', 'v', 'w'] -> x
['v', 'w', 'x'] -> z
['w', 'x', 'y'] -> z


In [18]:
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	print(seq_in, '->', seq_out)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

1 -> 2
2 -> 3
3 -> 4
4 -> 5
5 -> 6
6 -> 7
7 -> 8
8 -> 9
9 -> 0
0 -> a
a -> b
b -> c
c -> d
d -> e
e -> f
f -> g
g -> h
h -> i
i -> j
j -> k
k -> l
l -> m
m -> n
n -> o
o -> p
p -> q
q -> r
r -> s
s -> t
t -> u
u -> v
v -> w
w -> x
x -> y
y -> z


In [19]:
%%time
batch_size = 1
model = Sequential()
model.add(LSTM(32, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
for i in range(300):
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
	model.reset_states()
# summarize performance of the model
scores = model.evaluate(X, y, batch_size=batch_size, verbose=0)
model.reset_states()
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 85.71%
Wall time: 14.2 s


In [20]:
# demonstrate some model predictions
seed = [char_to_int[alphabet[0]]]
for i in range(0, len(alphabet)-1):
	x = numpy.reshape(seed, (1, len(seed), 1))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	print(int_to_char[seed[0]], "->", int_to_char[index])
	seed = [index]
model.reset_states()
# demonstrate a random starting point
letter = "a"
index = char_to_int[letter]
seed = [index]
print("New start: ", seed, index)
for i in range(0, 5):
    x = numpy.reshape(seed, (1, len(seed), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    print(int_to_char[seed[0]], "->", index, int_to_char[index])
    seed = [index]
model.reset_states()
print(prediction)

1 -> 2
2 -> 3
3 -> 4
4 -> 5
5 -> 6
6 -> 7
7 -> 8
8 -> 9
9 -> 0
0 -> a
a -> b
b -> c
c -> d
d -> e
e -> f
f -> g
g -> h
h -> i
i -> j
j -> k
k -> l
l -> m
m -> n
n -> o
o -> p
p -> p
p -> q
q -> s
s -> t
t -> t
t -> u
u -> w
w -> x
x -> x
x -> z
New start:  [10] 10
a -> 1 2
2 -> 2 3
3 -> 2 3
3 -> 3 4
4 -> 4 5
[[7.4861432e-06 5.3270726e-04 1.4103095e-02 1.9836536e-01 5.9980923e-01
  1.5112400e-01 1.7632097e-02 6.1145336e-03 2.5556590e-03 7.0959999e-05
  1.5699474e-06 3.9794190e-06 2.2343955e-05 1.6311454e-06 3.7749178e-06
  2.4852259e-05 3.1031217e-04 2.2651679e-03 8.2667515e-04 1.8670507e-04
  1.1089261e-03 1.4299252e-04 4.6600634e-04 2.3609668e-04 6.1529089e-04
  4.3131455e-04 3.3702303e-04 2.6897556e-04 2.7037258e-04 1.5115122e-04
  5.1723844e-05 3.5509755e-04 1.0464609e-04 3.3588937e-04 4.0960210e-04
  7.5278356e-04]]


In [21]:
from keras.preprocessing.sequence import pad_sequences

In [22]:
# prepare the dataset of input to output pairs encoded as integers
num_inputs = 1000
max_len = 5
dataX = []
dataY = []
for i in range(num_inputs):
    start = numpy.random.randint(len(alphabet)-2)
    end = numpy.random.randint(start, min(start+max_len,len(alphabet)-1))
    sequence_in = alphabet[start:end+1]
    sequence_out = alphabet[end + 1]
    dataX.append([char_to_int[char] for char in sequence_in])
    dataY.append(char_to_int[sequence_out])
    if i<10:
        print(sequence_in, '->', sequence_out)

def -> g
u -> v
stuvw -> x
567 -> 8
xy -> z
hijkl -> m
56 -> 7
67890 -> a
hijk -> l
6 -> 7


In [23]:
# convert list of lists to array and pad sequences if needed
X = pad_sequences(dataX, maxlen=max_len, dtype='float32')
# reshape X to be [samples, time steps, features]
X = numpy.reshape(X, (X.shape[0], max_len, 1))
# normalize
#X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(X[:4])

[[[ 0.]
  [ 0.]
  [13.]
  [14.]
  [15.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [30.]]

 [[28.]
  [29.]
  [30.]
  [31.]
  [32.]]

 [[ 0.]
  [ 0.]
  [ 4.]
  [ 5.]
  [ 6.]]]


In [24]:
%%time
# create and fit the model
batch_size = 8
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], 1)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, batch_size=batch_size, verbose=0)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 98.20%
Wall time: 2min 3s


In [24]:
# demonstrate some model predictions
for i in range(20):
	pattern_index = numpy.random.randint(len(dataX))
	pattern = dataX[pattern_index]
	x = pad_sequences([pattern], maxlen=max_len, dtype='float32')
	x = numpy.reshape(x, (1, max_len, 1))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(seq_in, "->", result)

['t'] -> u
['r', 's', 't'] -> u
['4', '5', '6'] -> 7
['x', 'y'] -> z
['i'] -> j
['e'] -> f
['3', '4', '5'] -> 6
['w', 'x'] -> y
['v'] -> w
['o', 'p', 'q', 'r'] -> s
['3', '4', '5'] -> 6
['w'] -> x
['6', '7', '8'] -> 9
['4', '5'] -> 6
['t', 'u', 'v', 'w'] -> x
['u', 'v'] -> w
['u', 'v', 'w', 'x', 'y'] -> z
['g', 'h', 'i', 'j', 'k'] -> l
['q'] -> r
['o'] -> p
